# 02 - Preprocesamiento y Feature Engineering

**Objetivo:** En este notebook, preparamos el dataset para el entrenamiento de modelos de Machine Learning. Las tareas incluyen: selección de características basada en el EDA, separación de datos en conjuntos de entrenamiento y prueba, y escalado de las características numéricas.

## 1. Carga de Datos e Importaciones

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [11]:
# Cargar el dataset optimizado
df = pd.read_parquet(r'../data/processed/cic_ids_2017_optimized.parquet')

## 2. Selección de Características (Feature Selection)

Basado en el Análisis Exploratorio de Datos (EDA), tomaremos decisiones sobre qué características conservar.

In [12]:
# Selección Manual (Basada en Correlación)
# En el EDA, vimos una correlación muy alta (0.89) entre 'Flow Duration' y 'Flow IAT Mean'.
# Decidimos eliminar 'Flow IAT Mean' para reducir la redundancia.
df.drop(columns = ['Flow IAT Mean'], inplace = True)

## 3. Separación de Datos

Separamos nuestras variables predictoras (**X**) de nuestra variable objetivo (**y**).

In [13]:
X = df.drop('Label', axis = 1)
y = df['Label']

## 4. División en Conjuntos de Entrenamiento y Prueba (Train/Test Split)

Dividimos el dataset. Es **crucial** hacer esto **antes** de cualquier otro preprocesamiento (como el escalado) para evitar la fuga de datos (data leakage). Usamos **stratify = y** para asegurar que la proporción de clases sea la misma en ambos conjuntos, lo cual es vital para datasets desbalanceados.

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

## 5. Selección de Características Automática (SelectFromModel)

Ahora usamos un modelo **RandomForest** como "juez" para que nos ayude a seleccionar las características más importantes de forma automática.

In [15]:
# Creamos el modelo que actuará como selector
selector_model = RandomForestClassifier(n_estimators = 50, random_state = 42, n_jobs= -1)

# Creamos el objeto selector, que elegirá las características con importancia > a la mediana
selector = SelectFromModel(estimator = selector_model, threshold = 'median')

# Entrenamos el selector
selector.fit(X_train, y_train)

# Obtenemos los nombres de las columnas seleccionadas
selected_features_mask = selector.get_support()
selected_features_names = X_train.columns[selected_features_mask]

# Transformamos nuestros conjuntos para quedarnos solo con las columnas seleccionadas
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Reportamos los resultados
print("\nProceso de selección completado.")
print("Número original de características:", X_train.shape[1])
print("Número de características seleccionadas:", X_train_selected.shape[1])
print("\nCaracterísticas seleccionadas:")
print(selected_features_names.tolist())

# Convertimos los arrays de numpy de vuelta a DataFrames de Pandas
X_train_selected = pd.DataFrame(X_train_selected, columns = selected_features_names)
X_test_selected = pd.DataFrame(X_test_selected, columns = selected_features_names)


Proceso de selección completado.
Número original de características: 77
Número de características seleccionadas: 39

Características seleccionadas:
['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Max', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'PSH Flag Count', 'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Fwd Header Length.1', 'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward']

## 6. Escalado de Características Numéricas (Scaling)

Finalmente, escalamos los datos para que todas las características tengan una media de 0 y una desviación estándar de 1. Esto es crucial para el rendimiento de muchos modelos.

In [16]:
# Creamos el escalador
scaler = StandardScaler()

# "Ajustamos" el escalador SÓLO con los datos de entrenamiento para que aprenda la media y desviación estándar
scaler.fit(X_train_selected)

# "Transformamos" tanto el conjunto de entrenamiento como el de prueba
X_train_scaled = scaler.transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

## Conclusión

En este punto, tenemos nuestros datos listos para el modelado:
- **X_train_scaled** y **y_train** para entrenar nuestros modelos.
- **X_test_scaled** y **y_test** para evaluarlos de forma honesta.